# Detector de sueño

En este notebook se describe la funcionalidad necesaria para crear un detector de sueño. El notebook está pensado para trabajar con la webcam del ordenador, pero también se puede utilizar una Raspberry Pi. El notebook está basado en varios [tutoriales de Pyimagesearch](https://www.pyimagesearch.com/2017/05/08/drowsiness-detection-opencv/). 

## 1. Detección de puntos clave faciales

Para este trabajo, vamos a utilizar los puntos clave faciales. Esta taréa consta de dos pasos:
1. Localizar la cara en la imagen.
2. Detectar las estructuras clave faciales en la cara.

La detección de la cara puede llevarse a cabo de múltiples maneras, por ejemplo con una combinación de [descriptores HOG y de un detector SVM](https://www.pyimagesearch.com/2014/11/10/histogram-oriented-gradients-object-detection/) o usando algoritmos de deep learning para localización de caras. 

Para el segundo paso, hay una variedad de detectores de puntos clave faciales que intentan localizar y etiquetar las siguientes regiones:
- boca,
- ceja izquierda y ceja derecha,
- ojo izquierdo y ojo derecho,
- naríz, y 
- mentón

Nosotros utilizaremos el detector de puntos clave faciales de la librería dlib que implementa un algoritmo presentado en [2014](https://pdfs.semanticscholar.org/d78b/6a5b0dcaa81b1faea5fb0000045a62513567.pdf).

Este detector de puntos clave estima la posición de 68 coordenadas (pares(x,y)) que asignan estructuras faciales de la cara. Los índices de las 68 coordenadas se pueden visualizar en la siguiente imagen. 

<img src="images/landmarks.jpg">

En esta primera parte veremos cómo detectar esos puntos en una imagen. 




Comenzamos cargando las librerías necesarias. Es posible que necesites instalar algunas de estas librerías. La mayoría las puedes instalar usando `pip`. La instalación de OpenCV y dlib está disponible en los siguientes enlaces:
- [Instalación OpenCV](https://www.pyimagesearch.com/2016/10/24/ubuntu-16-04-how-to-install-opencv/).
- [Instalación dlib](https://www.pyimagesearch.com/2017/03/27/how-to-install-dlib/).

In [ ]:
from imutils import face_utils
import numpy as np
import imutils
import dlib
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

En la siguiente línea se especifíca el path donde está el modelo pre-entrenado para detectar puntos clave faciales. En principio no deberías necesitar modificar dicho path ya que el modelo está disponible en la carpeta `models`, puede que sí que necesites descomprimir el archivo que se encuentra en dicha carpeta.  

In [ ]:
shape_predictor = "models/shape_predictor_68_face_landmarks.dat"

Inicializamos el detector de caras y creamos el predictor de puntos clave faciales.

In [ ]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(shape_predictor)

A continuación definimos una función que toma como parámetros el path de una imagen, el detector y el predictor y genera una imagen con la detección de la cara y de los puntos clave. El proceso es el siguiente:
1. Lee la imagen.
2. Reescala la imagen. 
3. Convierte la imagen a escala de grises.
4. Detecta las caras que hay en la imagen.
5. Para cada una de las caras de la imagen:
    1. Detecta los puntos clave de la imagen.
    2. Convierte los rectángulos de dlib a rectángulos de OpenCV.
    3. Muestra los puntos de la cara.
    
Comprueba que entiendes que hace cada línea del siguiente código. En caso de dudas visita la página de [Pyimagesearch](https://www.pyimagesearch.com/2017/04/03/facial-landmarks-dlib-opencv-python/).

In [ ]:
def detecta_puntos_clave(imagePath,detector,predictor):
    image = cv2.imread(imagePath)
    image = imutils.resize(image,width=500)
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    rects = detector(gray,1)
    
    for (i,rect) in enumerate(rects):
        shape = predictor(gray,rect)
        shape = face_utils.shape_to_np(shape)
        (x,y,w,h) = face_utils.rect_to_bb(rect)
        cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
        cv2.putText(image, "Cara #{}".format(i + 1), (x - 10, y - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        for (x, y) in shape:
            cv2.circle(image, (x, y), 1, (0, 0, 255), -1)
    # Conversion de la imagen para mostrarla en el notebook
    img2 = image[:,:,::-1]
    plt.imshow(img2)
    plt.show()

Puedes probar el código anterior con las imágenes de prueba disponible en el proyecto o con tus propias imágenes. Todas las imágenes de muestra tienen licencia que permiten su reutilización no comercial.

In [ ]:
image1Path = "muestra/image1.jpeg"
detecta_puntos_clave(image1Path,detector,predictor)

In [ ]:
image2Path = "muestra/imagen2.jpeg"
detecta_puntos_clave(image2Path,detector,predictor)

In [ ]:
image3Path = "muestra/image3.jpeg"
detecta_puntos_clave(image3Path,detector,predictor)

## 2. Detección de parpadeo

A continuación vamos a crear una aplicación que permite contar el número de parpadeos en un vídeo. Para ello vamos a utilizar una métrica llamada ratio de aspecto de los ojos (en inglés _eye aspect ratio_ o EAR) que fue introducido en [2016](http://vision.fe.uni-lj.si/cvww2016/proceedings/papers/05.pdf).

Las aproximaciones tradicionales para contar parpadéos suelen constar de tres pasos:
1. Localización de los ojos.
2. Aplicar un umbral para localizar el blanco de los ojos.
3. Determinar si la región blanca de los ojos desaparece por un periodo de tiempo.

El ratio de aspecto de los ojos, a partir de ahora EAR, involucra un cálculo sencillo basado en el ratio de distancias entre los puntos clave faciales de los ojos. Este método es rápido, eficiente y fácil de implementar. 


Ya hemos visto en la sección anterior cómo se pueden localizar puntos clave faciales que incluyen los ojos, las cejas, la nariz, las orejas y la boca. Esto significa que podemos extraer estructuras faciales concretas si conocemos los índices de esas partes. 

En lo que respecta a los ojos, solo estamos interesados en 6 coordenadas que comienzan en la esquina izquierda de un ojo y van en el sentido de las agujas del reloj alrededor del ojo. 

<img src="images/blink_detection_6_landmarks.jpg">

Basándonos en la imagen anterior se puede definir una relación entre el ancho y el alto de esas coordenadas. En concreto, la definición del EAR es:
$$EAR = \frac{||p_2-p_6||+||p_3-p_5||}{2||p_1-p_4||}$$

El numerador de la ecuación anterior calcula la distancia entre los puntos clave verticales, y el denominador, calcula la distancia entre los horizontales. El denominador se multiplica por 2 para comensar que sólo hay un conjunto de puntos horizontales pero 2 conjuntos de puntos verticales. 

¿Por qué es interesante esta ecuación? Es importante notar que mientras que los ojos están abiertos, el valor del EAR es aproximadamente constante, pero en cuanto los ojos parpadean, este valor tiende a cero rápidamente. Esto se puede ver en la siguiente imagen:

<img src="images/blink_detection_plot.jpg">

En la imagen superior izquierda, tenemos un ojo abierto cuyo EAR será constante a lo largo del tiempo. Por el contrario, en la imagen superior derecha este valor tiende a cero rápidamente. La parte inferior de la figura muestra un gráfico del EAR a lo largo del tiempo en un vídeo, como se puede apreciar hay un pico que indica que ha habido un parpadeo. 


Lo primero que tenemos que hacer para construir nuestro detector de parpadeos es cargar una serie de paquetes. Si no tienes alguno de los paquetes puedes instalarlo usando `pip`. Para acceder a un vídeo disponible en nuestro disco duro o a una webcam usaremos la librería `imutils`. 

In [ ]:
from scipy.spatial import distance as dist
from imutils.video import FileVideoStream
from imutils.video import VideoStream
import time
from IPython import display

Definimos la función que calcula el EAR a partir de los puntos clave de un ojo. Para ello se calcula la distancia Euclídea (utilizando la librería `scipy.spatial`) entre los puntos $p_2$ y $p_6$, entre los puntos $p_3$ y $p_5$, y entre los puntos $p_1$ y $p_4$ (recordar que en Python los índices de los vectores empiezan en 0).

In [ ]:
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

A continuación definimos dos constantes, una para indicar el valor mínimo de EAR para que se considere que el ojo está cerrado y el número de frames consecutivos que el ojo debe estar por debajo del valor mínimo de EAR para que se considere un parpadeo.

In [ ]:
UMBRAL_EAR = 0.3
MIN_N_FRAMES = 3

Definimos a continuación que cuenta el número de parpadeos. Para ello toma como parámetros un vídeo (en caso de no pasar el path de un vídeo como parámetro se utiliza la webcam), el detector de caras y el predictor de puntos clave faciales. En caso de que tengas dudas con el siguiente código puedes visitar la página de [PyimageSearch](https://www.pyimagesearch.com/2017/04/24/eye-blink-detection-opencv-python-dlib/). El proceso que se utiliza en esta función es el siguiente:
1. Detecta los puntos asociados con el ojo izquierdo y el ojo derecho.
2. Determina si se va a usar un vídeo o la webcam.
3. Mientras que no se haya acabado el vídeo se mide el EAR y se cuentan los parpadeos. 

In [ ]:
def contador_parpadeos(videoPath=None,detector=detector,predictor=predictor):
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
    if videoPath:
        vs = FileVideoStream(videoPath).start()
        fileStream = True
    else:
        vs = cv2.VideoCapture(0)
        fileStream = False
    time.sleep(1.0)
    contador = 0
    total = 0
    is_capturing, frame = vs.read()
    frame = frame[:,:,::-1]
    webcam_preview = plt.imshow(frame) 
    while True:
        try:
            if fileStream and not vs.more():
                break
            _,frame = vs.read()
            frame = imutils.resize(frame, width=450)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            rects = detector(gray, 0)
            for rect in rects:
                shape = predictor(gray, rect)
                shape = face_utils.shape_to_np(shape)
                leftEye = shape[lStart:lEnd]
                rightEye = shape[rStart:rEnd]
                leftEAR = eye_aspect_ratio(leftEye)
                rightEAR = eye_aspect_ratio(rightEye)
                ear = (leftEAR + rightEAR) / 2.0
                leftEyeHull = cv2.convexHull(leftEye)
                rightEyeHull = cv2.convexHull(rightEye)
                cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
                cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
                if ear < UMBRAL_EAR:
                    contador += 1
                else:
                    if contador>=MIN_N_FRAMES:
                        total+=1
                    contador = 0
                cv2.putText(frame, "Blinks: {}".format(total), (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                frame = frame[:,:,::-1] 
                webcam_preview.set_data(frame)
                plt.draw()
                display.clear_output(wait=True)
                display.display(plt.gcf())
                plt.pause(0.1)
        except KeyboardInterrupt:
            vs.release()

In [ ]:
contador_parpadeos()

## 3. Llegamos por fin a nuestro detector de sueño

La idea del detector de sueño es muy similar a la del contador de parpadeos. La única diferencia es el número de frames con un EAR por debajo de un umbral que tienen que pasar para considerar que alguien está dormido y no simplemente parpadeando. En nuestro caso, imponemos que si la persona del vídeo pasa más de 50 frames con los ojos cerrados, entonces se ha quedado dormida y emitiremos un sonido para despertarla.

Comenzamos cargando una librería que no servirá para emitir un sonido en caso de que el programa detecte que nos estamos quedando dormidos. Instala dicha librería usando `pip`.

In [ ]:
from threading import Thread
import playsound

A continuación definimos una función que produce un sonido de alarma utilizando un fichero de audio disponible en el disco duro. 

In [ ]:
def reproduce_alarma(path):
    playsound.playsound(path)

A continuación se muestra un ejemplo utilizando un sonido almacenado en la carpeta audio. 

In [ ]:
reproduce_alarma("audio/alarm.mp3")

Por último definimos nuestro detector de sueño. Pero primero modificamos la variable global `MIN_N_FRAMES` para indicar el número de frames que tienen que pasar para que consideremos que alguien está dormido.  

In [ ]:
MIN_N_FRAMES = 30

In [ ]:
def detecta_dormido(videoPath=None,sonido="audio/alarm.mp3",detector=detector,predictor=predictor):
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]
    if videoPath:
        vs = FileVideoStream(videoPath).start()
        fileStream = True
    else:
        vs = cv2.VideoCapture(0)
        fileStream = False
    time.sleep(1.0)
    contador = 0
    total = 0
    is_capturing, frame = vs.read()
    frame = frame[:,:,::-1]
    webcam_preview = plt.imshow(frame) 
    while True:
        try:
            if fileStream and not vs.more():
                break
            _,frame = vs.read()
            frame = imutils.resize(frame, width=450)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            rects = detector(gray, 0)
            for rect in rects:
                shape = predictor(gray, rect)
                shape = face_utils.shape_to_np(shape)
                leftEye = shape[lStart:lEnd]
                rightEye = shape[rStart:rEnd]
                leftEAR = eye_aspect_ratio(leftEye)
                rightEAR = eye_aspect_ratio(rightEye)
                ear = (leftEAR + rightEAR) / 2.0
                leftEyeHull = cv2.convexHull(leftEye)
                rightEyeHull = cv2.convexHull(rightEye)
                cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
                cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
                if ear < UMBRAL_EAR:
                    contador += 1
                    if contador>=MIN_N_FRAMES:
                        t = Thread(target=reproduce_alarma,args=(sonido,))
                        t.deamon = True
                        t.start()
                        cv2.putText(frame, "¡Te duermes!", (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                else:
                    contador=0
                cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                frame = frame[:,:,::-1]
                webcam_preview.set_data(frame)
                plt.draw()
                display.clear_output(wait=True)
                display.display(plt.gcf())
                plt.pause(0.1)
        except KeyboardInterrupt:
            vs.release()

Por último podemos probar nuestro detector de sueño.

In [ ]:
detecta_dormido()